In [1]:
from __future__ import print_function

import subprocess
import seaborn as sns
import pandas as pd
import DyMat

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import numpy as np
import math
from tqdm import tqdm
import os

import tempfile

In [11]:
%cd seesaw3_build/

/home/lukas/Projects/TrussFab/src/system_simulation/seesaw3_build


In [28]:
overrides = "startTime=0,stopTime=10,stepSize=0.02,outputFormat=mat,variableFilter=revLeft.*"

In [5]:
time subprocess.call(["./seesaw3", "-override springDamperParallel1.c='10"])

CPU times: user 705 µs, sys: 7.61 ms, total: 8.31 ms
Wall time: 26.6 ms


0

In [22]:
subprocess.run("./seesaw3 -outputPath={} -override={}".format('.', overrides, 100), shell=True, capture_output=True)

CompletedProcess(args='./seesaw3 -outputPath=. -override=startTime=0,stopTime=10,stepSize=0.2,outputFormat=mat,variableFilter=revLeft.*', returncode=0, stdout=b'LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.\nLOG_SUCCESS       | info    | The simulation finished successfully.\n', stderr=b'')

In [23]:
def getFreq(c=100):
    with tempfile.TemporaryDirectory(dir='.') as tempdir:
        subprocess.run("./seesaw3 -outputPath={} -override {},springDamperParallel1.c='{}'".format(tempdir, overrides, c), check=True, shell=True)
        d = DyMat.DyMatFile(tempdir + '/seesaw3_res.mat')
    return d['revLeft.phi']

def getFreqPlot(c=100):
    return pd.DataFrame(getFreq(c)).plot(title="C={}".format(c))

In [24]:
def range_metric(array: np.array):
    return array.ptp()

In [25]:
time getFreq(10)

CPU times: user 68.7 ms, sys: 12.1 ms, total: 80.8 ms
Wall time: 188 ms


array([0.        , 0.20699349, 0.98074964, 2.19795294, 3.3710424 ,
       4.10657144, 4.1858856 , 3.59318983, 2.73981659, 1.88125847,
       1.14243924, 0.66924463, 0.49170721, 0.37251106, 0.43385236,
       0.82880089, 1.6550358 , 2.61361065, 3.49131079, 4.02189251,
       3.93618554, 3.26810988, 2.23120403, 1.1800205 , 0.47385253,
       0.45127098, 0.91854868, 1.5432153 , 2.28641216, 2.93454861,
       3.21059005, 3.09544685, 2.75019373, 2.4039138 , 2.05279461,
       1.71360792, 1.41595573, 1.60331712, 2.3364836 , 2.7501428 ,
       2.19453405, 1.26617526, 0.76662296, 0.64641082, 0.8842344 ,
       1.38135182, 1.9940553 , 2.4265757 , 2.30052249, 1.87670712,
       2.00181237, 2.00181237])

In [29]:
interact(getFreqPlot, c=(0,20000,100))

interactive(children=(IntSlider(value=100, description='c', max=20000, step=100), Output()), _dom_classes=('wi…

<function __main__.getFreqPlot(c=100)>

In [ ]:
time getFreq(100)

In [ ]:
ranges = []

for c in tqdm(range(0, 20000, 100)):
    ranges.append(range_metric(getFreq(c)))
    
pd.DataFrame(ranges).plot()

In [ ]:
import time
import concurrent.futures

e = concurrent.futures.ThreadPoolExecutor(4)
s = range(0, 20000, 100)
calc = lambda c: range_metric(getFreq(c))

res = [x for x in tqdm(e.map(calc, s))]

In [ ]:
pd.DataFrame(res).plot()